Підготовка та аналіз даних
Лабораторна робота №2
ФБ-25 Кравченко Максим

Імпортуємо потрібні нам бібліотеки

In [3]:
import datetime
import requests
import urllib.request
import pandas as pd
import os


Скачуємо VHI індекси для всіх областей
Також додаємо в назву дату та час скачування

In [4]:
for ids in range(0, 27):
    ids += 1
    url = f"https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={ids}&year1=1981&year2=2024&type=Mean"
    response = requests.get(url)
    if response.status_code == 200:
        date_now = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        vhi_url = urllib.request.urlopen(url)
        file_name = f'vhi/vhi_id_{ids}_{date_now}.csv'
        out = open(file_name, 'wb')
        out.write(vhi_url.read())
        out.close()
        print(f"VHI from id {ids} was downloaded at {date_now}")
    else:
        print(f"Download complete")
        break

VHI from id 1 was downloaded at 2024-03-26_23-33-56
VHI from id 2 was downloaded at 2024-03-26_23-33-58
VHI from id 3 was downloaded at 2024-03-26_23-34-01
VHI from id 4 was downloaded at 2024-03-26_23-34-04
VHI from id 5 was downloaded at 2024-03-26_23-34-06
VHI from id 6 was downloaded at 2024-03-26_23-34-09
VHI from id 7 was downloaded at 2024-03-26_23-34-11
VHI from id 8 was downloaded at 2024-03-26_23-34-13
VHI from id 9 was downloaded at 2024-03-26_23-34-16
VHI from id 10 was downloaded at 2024-03-26_23-34-18
VHI from id 11 was downloaded at 2024-03-26_23-34-20
VHI from id 12 was downloaded at 2024-03-26_23-34-23
VHI from id 13 was downloaded at 2024-03-26_23-34-25
VHI from id 14 was downloaded at 2024-03-26_23-34-28
VHI from id 15 was downloaded at 2024-03-26_23-34-30
VHI from id 16 was downloaded at 2024-03-26_23-34-33
VHI from id 17 was downloaded at 2024-03-26_23-34-35
VHI from id 18 was downloaded at 2024-03-26_23-34-38
VHI from id 19 was downloaded at 2024-03-26_23-34-40
VH

Даємо імена стовбцям, зчитуємо файли у фрейм

In [5]:
folder = 'vhi'
files = os.listdir(folder)
ids = 0
df_all = pd.DataFrame()

for file_name in files:
    ids += 1
    headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'empty']
    df = pd.read_csv(f'{folder}/{file_name}', header=1, names=headers, skiprows=1)[:-1]
    df = df.drop(df.loc[df['VHI'] == -1].index)
    df['area'] = ids
    # print(df.head(), '\n')
    df_all = pd.concat([df_all, df], ignore_index=True)

# print(df_all, '\n')

Змінимо індекси областей

In [6]:
dict_areas = {1: 22, 2: 24, 3: 23, 4: 25, 5: 3, 6: 4, 7: 8, 8: 19, 9: 20, 10: 21, 11: 9, 13: 10, 14: 11, 15: 12,
              16: 13, 17: 15, 18: 14, 19: 16, 21: 17, 22: 18, 23: 6, 24: 1, 25: 2, 26: 7, 27: 5}
df_all["area"] = df_all["area"].replace(dict_areas)
df_all.to_csv(f'vhi/df_all.csv', index=False)

print(df_all)

       Year  Week    SMN     SMT    VCI    TCI    VHI  empty  area
0      1982   2.0  0.063  261.53  55.89  38.20  47.04    NaN    22
1      1982   3.0  0.063  263.45  57.30  32.69  44.99    NaN    22
2      1982   4.0  0.061  265.10  53.96  28.62  41.29    NaN    22
3      1982   5.0  0.058  266.42  46.87  28.57  37.72    NaN    22
4      1982   6.0  0.056  267.47  39.55  30.27  34.91    NaN    22
...     ...   ...    ...     ...    ...    ...    ...    ...   ...
57910  2024   8.0  0.134  281.83  62.33   3.92  33.13    NaN     5
57911  2024   9.0  0.147  283.80  61.79   4.57  33.18    NaN     5
57912  2024  10.0  0.158  285.63  59.96   6.30  33.13    NaN     5
57913  2024  11.0  0.164  286.56  56.53   9.83  33.18    NaN     5
57914  2024  12.0  0.175  287.65  55.86  13.89  34.87    NaN     5

[57915 rows x 9 columns]


Ряд VHI для області за вказаний рік, пошук екстремумів (min та max);

In [7]:
def vhi(area, year):
    return df_all[(df_all["area"] == area) & (df_all["Year"] == year)]['VHI']

def vhi_min(area, year):
    return df_all[(df_all["area"] == area) & (df_all["Year"] == year)]['VHI'].min()

def vhi_max(area, year):
    return df_all[(df_all["area"] == area) & (df_all["Year"] == year)]['VHI'].max()

df_all['Year'] = df_all['Year'].astype(int)

In [8]:
vhi(5, 2020)

57695    45.94
57696    45.28
57697    44.67
57698    44.62
57699    44.56
57700    44.91
57701    44.90
57702    45.07
57703    45.25
57704    45.00
57705    44.59
57706    45.10
57707    46.75
57708    48.09
57709    49.77
57710    52.60
57711    55.69
57712    57.93
57713    61.80
57714    63.49
57715    62.18
57716    59.98
57717    57.35
57718    53.68
57719    50.56
57720    47.67
57721    45.29
57722    43.91
57723    42.69
57724    41.22
57725    39.86
57726    37.98
57727    35.40
57728    32.74
57729    29.58
57730    25.81
57731    24.49
57732    25.05
57733    27.68
57734    32.59
57735    35.98
57736    36.08
57737    36.82
57738    35.40
57739    35.94
57740    37.39
57741    39.34
57742    41.37
57743    42.55
57744    42.43
57745    43.19
57746    44.57
Name: VHI, dtype: float64

In [9]:
vhi_min(5, 2005)

28.47

In [10]:
vhi_max(5, 2005)

67.33

Ряд VHI за вказаний діапазон років для вказаних областей;

In [11]:
def vhi_range(year_min, year_max, areas):
    if not isinstance(areas, list) or not areas:
        return print('Empty or not a list')
    return df_all[(df_all['Year'] >= year_min) & (df_all['Year'] <= year_max) & (df_all['area'].isin(areas))][['Year', 'VHI', 'area']]

In [12]:
vhi_range(1998, 2005, [1, 2, 3])

,Year,VHI,area
9381,1998,48.42,3
9382,1998,45.60,3
9383,1998,43.56,3
9384,1998,42.32,3
9385,1998,41.53,3
...,...,...,...
52672,2005,49.13,2
52673,2005,48.69,2
52674,2005,47.39,2
52675,2005,47.00,2


Роки, протягом яких екстремальні посухи торкнулися більше вказаного відсотка областей

In [23]:
def extreme_droughts(procent):
    df_drouhts = df_all[(df_all['VHI'] <= 20) & (df_all['VHI'] != -1)]
    grouped = df_drouhts.groupby('Year')['area'].nunique()
    result = grouped[grouped > (25 * procent / 100)].reset_index()
    return result

In [24]:
extreme_droughts(20)

,Year,area
0,1986,8
1,1993,10
2,2000,16
3,2007,7


Аналогічно для помірних посух

In [27]:
def moderate_droughts(procent):
    df_drouhts = df_all[(df_all['VHI'] >= 30) & (df_all['VHI'] <= 55) & (df_all['VHI'] != -1)]
    grouped = df_drouhts.groupby('Year')['area'].nunique()
    result = grouped[grouped > (25 * procent / 100)].reset_index()
    return result

In [28]:
moderate_droughts(20)

,Year,area
0,1982,25
1,1983,25
2,1984,25
3,1985,25
4,1986,25
5,1987,25
6,1988,25
7,1989,25
8,1990,25
9,1991,25
